In [ ]:
import pandas as pd # 假设已导入
import numpy as np # 假设已导入
import smogn # 这个库在您提供的噪声注入和Mixup代码中并未使用，如果只是为这两个方法，可以不导入
from pathlib import Path 
CURRENT_DIR = Path.cwd()
PROJECT_ROOT = CURRENT_DIR.parent
DATA_DIR = PROJECT_ROOT / "data"
OUTPUT_DIR = PROJECT_ROOT / "output"
# 设置文件路径 (请根据您的实际路径修改)
file_path =DATA_DIR / "development_set_selected_features.xlsx" # 确保这是开发集数据

# 加载数据
try:
    data_df = pd.read_excel(file_path)
except FileNotFoundError:
    print(f"错误：文件未找到，请检查路径：{file_path}")
    data_df = None 

if data_df is not None:
    print("原始数据信息：")
    data_df.info() # data_df.info() 本身会打印，不需要再套一个print
    print("\n原始数据前5行：")
    print(data_df.head())

    target_column = "Rowing distance" # 请确保这是您目标变量的准确列名
    
    # --- 修改开始 ---
    # CHO 不再是特别的 categorical_column，它应被视为数值型输入特征
    # 如果您没有其他真正的类别特征，可以将 categorical_column 设为 None 或空列表
    # categorical_column = "CHO" # <--- 注释掉或删除这行

    # 假设除了目标列，其余都是数值型输入特征 (现在CHO也是其中之一)
    numerical_input_columns = [col for col in data_df.columns if col != target_column]
    
    # 如果您确实有其他需要特殊处理的类别特征（非CHO），在这里定义它们：
    # 例如: other_categorical_cols = ["SomeOtherCategoricalFeature"]
    # 那么 numerical_input_columns 可以这样定义:
    # numerical_input_columns = [col for col in data_df.columns 
    #                            if col not in [target_column] + other_categorical_cols]
    # 为简化，我们假设除了目标变量，其他都是数值型（包括CHO）
    
    print(f"\n数值型输入特征 ({len(numerical_input_columns)}): {numerical_input_columns}")
    # print(f"类别特征: {categorical_column}") # 如果 categorical_column 被设为 None 或移除，这行也应调整
    if 'categorical_column' in locals() and categorical_column is not None and len(categorical_column) > 0:
        print(f"类别特征: {categorical_column}")
    else:
        print("无特殊指定的类别特征（CHO 已视为数值型）。")
    print(f"目标特征: {target_column}")
    # --- 修改结束 ---

In [ ]:
# 随机噪声注入 (Random Noise Injection)
import pandas as pd
import numpy as np

def augment_with_noise(df, numerical_cols, 
                       noise_level_fraction=0.05,
                       absolute_num_to_generate=None):
    augmented_data = []
    num_original_samples = len(df)

    if absolute_num_to_generate is None:
        print("警告：未指定absolute_num_to_generate，将不生成样本。如需生成，请设置此参数。")
        num_to_generate = 0
    else:
        num_to_generate = int(absolute_num_to_generate)

    if num_to_generate <= 0:
        if absolute_num_to_generate is not None :
             print("警告：要生成的样本数量必须为正数。")
             return pd.DataFrame(columns=df.columns)
        # 如果未明确指定生成，num_to_generate可能为0，此时不生成数据
    
    for _ in range(num_to_generate):
        original_sample_idx = np.random.randint(0, num_original_samples)
        original_sample = df.iloc[original_sample_idx].copy()
        augmented_sample = original_sample.copy()

        for col in numerical_cols:
            noise = np.random.normal(0, df[col].std() * noise_level_fraction)
            augmented_sample[col] = augmented_sample[col] + noise
            
            # --- 修改开始：应用WGAN-GP类似的clipping逻辑 ---
            original_min = df[col].min()
            original_max = df[col].max()
            col_range = original_max - original_min

            if col_range == 0: 
                clip_min_for_col = original_min
                clip_max_for_col = original_max
            else:
                clip_min_for_col = original_min - 0.01 * col_range
                clip_max_for_col = original_max + 0.01 * col_range
            
            if col in ['CHO', 'PRO']: 
                clip_min_for_col = max(0, clip_min_for_col)

            augmented_sample[col] = np.clip(augmented_sample[col], clip_min_for_col, clip_max_for_col)
            # --- 修改结束 ---
            
        augmented_data.append(augmented_sample)

    if not augmented_data:
        return pd.DataFrame(columns=df.columns)
        
    return pd.DataFrame(augmented_data, columns=df.columns)

# --- 数据加载和调用部分保持不变，确保 data_df 和 numerical_input_columns 已正确定义 ---
# (示例调用)
# if data_df is not None:
#     noise_augmented_df_abs = augment_with_noise(
#         df=data_df,
#         numerical_cols=numerical_input_columns, 
#         noise_level_fraction=0.05,
#         absolute_num_to_generate=2000 
#     )
#     # ... 后续打印和保存 ...

# --- 数据加载部分 (假设您已经正确加载了 data_df 和 numerical_input_columns) ---
# 确保 data_df 是您的 development_set_selected_features.xlsx
# 确保 numerical_input_columns 包含了 'CHO' 并且不包含 target_column

# --- 如何调用 (假设 data_df 和 numerical_input_columns 已按新方式定义) ---
# 示例：确保您在调用前已经正确加载和定义了 data_df 和 numerical_input_columns
# file_path = r"C:\Path\To\Your\development_set_selected_features.xlsx"
# try:
#     data_df = pd.read_excel(file_path)
#     target_column = "Rowing distance"
#     numerical_input_columns = [col for col in data_df.columns if col != target_column] # CHO 包含在内
# except FileNotFoundError:
#     print(f"错误：文件未找到 {file_path}")
#     data_df = None

if data_df is not None:
    print("\n--- 随机噪声注入 (绝对数量并保存) ---")
    noise_augmented_df_abs = augment_with_noise(
        df=data_df,
        numerical_cols=numerical_input_columns, 
        noise_level_fraction=0.05,
        absolute_num_to_generate=2000 
    )
    
    print(f"\n通过绝对数量设置，生成了 {len(noise_augmented_df_abs)} 条噪声增强样本。") # 这行移到函数调用之后
    if not noise_augmented_df_abs.empty:
        print("噪声增强数据前5行 (绝对数量):")
        print(noise_augmented_df_abs.head())

        try:
            output_excel_path_noise_abs = DATA_DIR / "augmented_data_noise_abs_2000.xlsx" 
            noise_augmented_df_abs.to_excel(output_excel_path_noise_abs, index=False)
            print(f"噪声增强数据 (绝对数量) 已保存到: {output_excel_path_noise_abs}")
        except Exception as e:
            print(f"保存噪声增强数据 (绝对数量) 到Excel失败: {e}")
    elif absolute_num_to_generate is not None and absolute_num_to_generate > 0 : # 如果明确要求生成但结果为空
        print("错误或警告：要求生成增强数据，但结果为空。请检查函数逻辑或输入。")

In [ ]:
# Mixup
import pandas as pd
import numpy as np

def augment_with_mixup(df, numerical_cols, target_col, 
                       alpha=0.2,
                       absolute_num_to_generate=None,
                       other_categorical_cols_list=None): 
    
    augmented_data = []
    num_original_samples = len(df)

    if absolute_num_to_generate is not None:
        num_to_generate = int(absolute_num_to_generate)
    else:
        print("警告：未指定 absolute_num_to_generate，将不生成样本。如需生成，请设置此参数。")
        num_to_generate = 0

    if num_to_generate <= 0:
        if absolute_num_to_generate is not None : 
             print("警告：要生成的样本数量必须为正数。")
             return pd.DataFrame(columns=df.columns)

    if num_original_samples < 2 and num_to_generate > 0 :
        print("警告：Mixup至少需要2个原始样本才能进行。")
        return pd.DataFrame(columns=df.columns)

    X_numerical_df = df[numerical_cols]
    y_series = df[target_col]
    
    X_other_categorical_df = None
    valid_other_cat_cols = [] # 确保此变量在后续循环中可用
    if other_categorical_cols_list and len(other_categorical_cols_list) > 0:
        valid_other_cat_cols = [col for col in other_categorical_cols_list if col in df.columns]
        if len(valid_other_cat_cols) > 0:
            X_other_categorical_df = df[valid_other_cat_cols]
        else:
            print(f"警告：提供的 other_categorical_cols_list 中的列均不在DataFrame中。")

    for _ in range(num_to_generate):
        idx1, idx2 = np.random.choice(num_original_samples, 2, replace=False)
        
        sample1_X_num, sample1_y = X_numerical_df.iloc[idx1], y_series.iloc[idx1]
        sample2_X_num, sample2_y = X_numerical_df.iloc[idx2], y_series.iloc[idx2]

        lam = np.random.beta(alpha, alpha)
        
        mixed_numerical_X_series = lam * sample1_X_num + (1 - lam) * sample2_X_num
        mixed_y = lam * sample1_y + (1 - lam) * sample2_y
        
        augmented_sample_dict = mixed_numerical_X_series.to_dict()
        augmented_sample_dict[target_col] = mixed_y

        if X_other_categorical_df is not None and not X_other_categorical_df.empty:
            sample1_X_other_cat = X_other_categorical_df.iloc[idx1]
            sample2_X_other_cat = X_other_categorical_df.iloc[idx2]
            for cat_col_name in valid_other_cat_cols: # 使用验证过的列名
                mixed_cat_val = sample1_X_other_cat[cat_col_name] if np.random.rand() > 0.5 else sample2_X_other_cat[cat_col_name]
                augmented_sample_dict[cat_col_name] = mixed_cat_val
        
        # --- 修改开始：应用WGAN-GP类似的clipping逻辑 ---
        for col_name in numerical_cols: 
            if col_name in augmented_sample_dict: 
                original_min = df[col_name].min()
                original_max = df[col_name].max()
                col_range = original_max - original_min
                
                if col_range == 0:
                    clip_min_for_col = original_min
                    clip_max_for_col = original_max
                else:
                    clip_min_for_col = original_min - 0.01 * col_range
                    clip_max_for_col = original_max + 0.01 * col_range
                
                if col_name in ['CHO', 'PRO']: 
                    clip_min_for_col = max(0, clip_min_for_col) 
                
                augmented_sample_dict[col_name] = np.clip(augmented_sample_dict[col_name], clip_min_for_col, clip_max_for_col)
        # --- 修改结束 ---

        augmented_data.append(augmented_sample_dict)

    if not augmented_data:
        return pd.DataFrame(columns=df.columns)
        
    # 确保返回的DataFrame列顺序与原始df一致
    # 如果 augmented_data 包含所有原始列，则 df.columns 是合适的
    # 如果 augmented_data 中的字典可能不全（例如，如果 other_categorical_cols_list 为空且df有这些列），
    # 或者顺序不一致，最好明确指定顺序。
    # 但由于 augmented_sample_dict 是基于 mixed_numerical_X_series.to_dict() 和 target_col,
    # 以及 valid_other_cat_cols 构建的，它应包含所有参与的列。
    # 为了安全起见，如果df.columns包含了所有可能生成的列且顺序是你想要的，可以用它。
    # 如果担心顺序或列不全，可以构建一个包含所有df列名的列表，然后用 reindex。
    # 但通常，如果 augmented_sample_dict 的键是正确的，直接用 df.columns 应该可以。
    final_df = pd.DataFrame(augmented_data)
    # 确保所有原始列都存在，如果增强过程中某些列没有被加入字典，用NaN填充 (通常不应发生)
    # 并按原始顺序排列
    return final_df.reindex(columns=df.columns)


# --- 数据加载和调用部分保持不变，确保 data_df, numerical_input_columns, target_column, other_true_categorical_cols 已正确定义 ---
# (示例调用)
# if data_df is not None:
#     mixup_augmented_df_abs = augment_with_mixup(
#         df=data_df,
#         numerical_cols=numerical_input_columns, 
#         target_col=target_column,
#         other_categorical_cols_list=other_true_categorical_cols, 
#         alpha=0.2,
#         absolute_num_to_generate=2000  
#     )
#     # ... 后续打印和保存 ...


# --- 数据加载部分 (假设您已经正确加载了 data_df, numerical_input_columns, target_column) ---
# file_path = r"C:\Path\To\Your\development_set_selected_features.xlsx"
# try:
#     data_df = pd.read_excel(file_path)
#     target_column = "Rowing distance"
#     # 假设 CHO 包含在 numerical_input_columns 中
#     numerical_input_columns = [col for col in data_df.columns if col != target_column] 
#     # 定义其他真实类别特征（如果存在）
#     other_true_categorical_cols = [] # 例如: ["Gender", "SportType"]
# except FileNotFoundError:
#     print(f"错误：文件未找到 {file_path}")
#     data_df = None


# --- 如何调用 (假设 data_df 和 numerical_input_columns 已按新方式定义) ---
if data_df is not None:
    print("\n--- Mixup (绝对数量并保存) ---")
    # 假设没有其他真实类别特征，或者您已定义 other_true_categorical_cols
    # 确保这里的 other_true_categorical_cols 与函数内期望的 other_categorical_cols_list 一致
    other_true_categorical_cols = [] # 例如，如果真的没有其他类别列了
    # 如果有，例如： other_true_categorical_cols = ["SomeOtherCat"]

    mixup_augmented_df_abs = augment_with_mixup(
        df=data_df,
        numerical_cols=numerical_input_columns, 
        target_col=target_column,
        other_categorical_cols_list=other_true_categorical_cols, 
        alpha=0.2,
        absolute_num_to_generate=2000  
    )
    
    print(f"\n通过绝对数量设置，生成了 {len(mixup_augmented_df_abs)} 条Mixup增强样本。") # 移到函数调用后
    if not mixup_augmented_df_abs.empty:
        print("Mixup增强数据前5行 (绝对数量):")
        print(mixup_augmented_df_abs.head())

        try:
            output_excel_path_mixup_abs = DATA_DIR / "augmented_data_mixup_abs_2000.xlsx"
            mixup_augmented_df_abs.to_excel(output_excel_path_mixup_abs, index=False)
            print(f"Mixup增强数据 (绝对数量) 已保存到: {output_excel_path_mixup_abs}")
        except Exception as e:
            print(f"保存Mixup增强数据 (绝对数量) 到Excel失败: {e}")
    elif absolute_num_to_generate is not None and absolute_num_to_generate > 0:
        print("错误或警告：要求生成增强数据，但结果为空。请检查函数逻辑或输入。")